<a href="https://colab.research.google.com/github/ThousandAI/Application-of-AI/blob/main/class02/class02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Autograd

In [1]:
import torch
import torch.nn as nn

In [2]:
# grad = True
tensor = torch.rand(3, requires_grad=True)
print(tensor)
y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z = z.mean()
print(z)
z.backward() # dz/d(tensor)
print(tensor.grad) # 2*tensor + 12

tensor([0.7141, 0.2197, 0.2249], requires_grad=True)
tensor([6.7141, 6.2197, 6.2249], grad_fn=<AddBackward0>)
tensor([137.2366, 118.0530, 118.2496], grad_fn=<AddBackward0>)
tensor(124.5130, grad_fn=<MeanBackward0>)
tensor([13.4282, 12.4393, 12.4499])


In [3]:
# grad = False
tensor = torch.rand(3, requires_grad=False)
print(tensor)

y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z = z.mean()
print(z)
z.backward() # dz/d(tensor)

tensor([0.4231, 0.3882, 0.9385])
tensor([6.4231, 6.3882, 6.9385])
tensor([125.7686, 124.4277, 146.4293])
tensor(132.2085)


RuntimeError: ignored

In [4]:
# not scalar
tensor = torch.tensor([0.37, 0.58, 0.33], requires_grad=True)
print(tensor)

y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z.backward()

tensor([0.3700, 0.5800, 0.3300], requires_grad=True)
tensor([6.3700, 6.5800, 6.3300], grad_fn=<AddBackward0>)
tensor([123.7307, 131.8892, 122.2067], grad_fn=<AddBackward0>)


RuntimeError: ignored

In [5]:
# grad 
tensor = torch.rand(3, requires_grad = True)
print(tensor)
y = tensor.detach()
print(y)

tensor([0.2025, 0.6930, 0.7874], requires_grad=True)
tensor([0.2025, 0.6930, 0.7874])


In [6]:
# grad 
with torch.no_grad():
  y = tensor + 2
  print(y)

tensor([2.2025, 2.6930, 2.7874])


In [7]:
# accumulated gradient
weights = torch.tensor([2., 3., 5., 7.], requires_grad=True)

for epoch in range(5):
  outputs = (3*weights).sum()
  outputs.backward()

  print(weights.grad)

tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])
tensor([9., 9., 9., 9.])
tensor([12., 12., 12., 12.])
tensor([15., 15., 15., 15.])


In [8]:
# empty gradient
weights = torch.tensor([2., 3., 5., 7.], requires_grad=True)

for epoch in range(5):
  outputs = (3*weights).sum()
  outputs.backward()

  print(weights.grad)

  weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


## toy example

In [9]:
# toy example
x = torch.tensor([[1,-1], [2,3], [5,2]], dtype=torch.float32) # 3x2
y = torch.tensor([[1],[0],[1]], dtype=torch.float32)

w1 = torch.rand(2,3, requires_grad=True)
w2 = torch.rand(3,3, requires_grad=True)
w3 = torch.rand(3,2, requires_grad=True)
w4 = torch.rand(2,1, requires_grad=True)
relu = nn.ReLU()
sigmoid = nn.Sigmoid()
bce = nn.BCELoss()

def forward(inputs):
    inputs = torch.matmul(inputs, w1)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w2)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w3)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w4)
    outputs = sigmoid(inputs)
    return outputs

# loss
def loss(y_true, y_pred):
    return bce(y_pred, y_true)


learning_rate = 0.01
epochs = 1000

for epoch in range(epochs):
    # forward pass
    y_hat = forward(inputs=x)

    # loss
    bce_loss = loss(y_true=y, y_pred=y_hat)

    # backward loss
    bce_loss.backward()

    # update weights
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        w3 -= learning_rate * w3.grad
        w4 -= learning_rate * w4.grad

    # zero gradients
    w1.grad.zero_()
    w2.grad.zero_()
    w3.grad.zero_()
    w4.grad.zero_()

    if epoch % 5 == 0:
        print(f"epoch {epoch + 1}: \nw1 = {w1}\n w2 = {w2}\n w3 = {w3}\n w4 = {w4}, loss = {bce_loss:.8f}")

epoch 1: 
w1 = tensor([[0.5637, 0.5597, 0.0216],
        [0.2780, 0.3940, 0.9003]], requires_grad=True)
 w2 = tensor([[0.6717, 0.3724, 0.4127],
        [0.4240, 0.8027, 0.4843],
        [0.2891, 0.6879, 0.4562]], requires_grad=True)
 w3 = tensor([[0.5246, 0.6724],
        [0.4000, 0.5708],
        [0.2727, 0.7777]], requires_grad=True)
 w4 = tensor([[0.5821],
        [0.6666]], requires_grad=True), loss = 2.76114726
epoch 6: 
w1 = tensor([[ 0.5391,  0.5322, -0.0065],
        [ 0.2263,  0.3361,  0.8581]], requires_grad=True)
 w2 = tensor([[0.6513, 0.3560, 0.3945],
        [0.3992, 0.7828, 0.4621],
        [0.2582, 0.6632, 0.4286]], requires_grad=True)
 w3 = tensor([[0.4988, 0.6435],
        [0.3617, 0.5278],
        [0.2458, 0.7475]], requires_grad=True)
 w4 = tensor([[0.5191],
        [0.5608]], requires_grad=True), loss = 1.96744168
epoch 11: 
w1 = tensor([[ 0.5202,  0.5114, -0.0277],
        [ 0.1856,  0.2912,  0.8261]], requires_grad=True)
 w2 = tensor([[0.6365, 0.3447, 0.3815],
   